<a href="https://colab.research.google.com/github/hamidzangiabadi/LSTM-HHO-IoMT-Colab/blob/colab/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import random
import math
import time
import cupy as cp
import time

# Implement a simple HHO algorithm

Description of HHO:



Useful functions

In [ ]:
# the function to optimize
def sphere_function(x):
    return cp.sum(x**2)

def Levy(dim):
    beta = 1.5
    sigma = ( math.gamma(1+beta) * math.sin(math.pi*beta/2) /
             ( math.gamma((1+beta)/2) * beta * 2**((beta-1)/2) ) )**(1/beta)
    u = 0.01 * np.random.randn(dim) * sigma
    v = np.random.randn(dim)
    zz = np.power(np.abs(v), (1/beta))
    step = u / zz
    return step

def initialize_population(pop_size, dim, lb, ub):
    return lb + np.random.rand(pop_size, dim) * (ub - lb)


# Levy flight for GPU
def levy_gpu(dim, beta=1.5):
    sigma = ( math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
              ( math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2) ) ) ** (1 / beta)
    u = 0.01 * np.random.randn(dim) * sigma
    v = np.random.randn(dim)
    step = u / (np.abs(v) ** (1 / beta))
    return cp.asarray(step, dtype=cp.float32)



# Compressed HHO and Simple HHO

In Compressed version I used Levy flight also but optimize the functions and also run the method on GPU

Simple HHO

In [ ]:
def HHO(objf, lb, ub, dim, SearchAgents_no, max_iter):

    # Rabbit position (best solution so far)
    Rabbit_Location = np.zeros(dim)
    Rabbit_Energy = float("inf")

    # Hawks positions
    X = initialize_population(SearchAgents_no, dim, lb, ub)

    # Convergence curve
    convergence_curve = np.zeros(max_iter)

    print(f"HHO is now tackling '{objf.__name__}'")
    timer_start = time.time()
    for t in range(max_iter):
        # Initilization
        for i in range(SearchAgents_no):
            X[i, :] = np.clip(X[i, :], lb, ub)
            fitness = objf(X[i, :])
            if fitness < Rabbit_Energy:
                Rabbit_Energy = fitness
                Rabbit_Location = X[i, :].copy()

        E1 = 2 * (1 - t / max_iter)

        # Hawks Movements
        for i in range(SearchAgents_no):
            E0 = 2 * random.random() - 1
            Escaping_Energy = E1 * E0

            if abs(Escaping_Energy) >= 1:
                # Exploration Eq.1
                q = random.random()
                if q >= 0.5:
                    r1 = random.random()
                    r2 = random.random()
                    rand_idx = math.floor(SearchAgents_no * random.random())
                    X_rand = X[rand_idx, :]
                    X[i, :] = X_rand - r1 * abs(X_rand - 2 * r2 * X[i, :])
                else:
                    r3 = random.random()
                    r4 = random.random()
                    X_mean = np.mean(X, axis=0)
                    random_point = lb + r4 * (ub - lb)
                    X[i, :] = (Rabbit_Location - X_mean) - r3 * (random_point)
            else:
                # Exploitation Eq.4-11
                r = random.random()
                Jump_Strength = 2 * (1 - random.random())

                if r >= 0.5 and abs(Escaping_Energy) >= 0.5:
                    DX = Rabbit_Location - X[i, :]
                    X[i, :] = DX - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - X[i, :])

                if r >= 0.5 and abs(Escaping_Energy) < 0.5:
                    X[i, :] = Rabbit_Location - Escaping_Energy * abs(Rabbit_Location - X[i, :])

                if r < 0.5 and abs(Escaping_Energy) >= 0.5:
                    X1 = Rabbit_Location - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - X[i, :])
                    if objf(X1) < objf(X[i, :]):
                        X[i, :] = X1.copy()
                    else:
                        X2 = X1 + np.multiply(np.random.randn(dim), Levy(dim))
                        if objf(X2) < objf(X[i, :]):
                            X[i, :] = X2.copy()

                if r < 0.5 and abs(Escaping_Energy) < 0.5:
                    X1 = Rabbit_Location - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - np.mean(X, axis=0))
                    if objf(X1) < objf(X[i, :]):
                        X[i, :] = X1.copy()
                    else:
                        X2 = X1 + np.multiply(np.random.randn(dim), Levy(dim))
                        if objf(X2) < objf(X[i, :]):
                            X[i, :] = X2.copy()

        # Diversity restore
        for i in range(SearchAgents_no):
            X[i, :] = np.clip(X[i, :], lb, ub)
            fitness = objf(X[i, :])
            if fitness < Rabbit_Energy:
                Rabbit_Energy = fitness
                Rabbit_Location = X[i, :].copy()

        convergence_curve[t] = Rabbit_Energy
        print(f"Iteration {t+1}/{max_iter} | Best fitness: {Rabbit_Energy:.6f}")

    timer_end = time.time()
    execution_time = timer_end - timer_start

    return {
        "best_fitness": Rabbit_Energy,
        "best_position": Rabbit_Location,
        "convergence_curve": convergence_curve,
        "execution_time": execution_time
    }


Optimized Adaptive HHO

In [ ]:

def compressed_hho_adaptive(fitness_func, lb, ub, dim, N, max_iter,
                             stagnation_limit=20, max_reset_rate=0.5):
    # Inilization of the population
    X = cp.random.uniform(lb, ub, (N, dim))  # Hawks Population

    fit = cp.array([fitness_func(cp.asnumpy(X[i])) for i in range(N)])  # Initial fitness

    best_idx = int(cp.argmin(fit))
    best_pos = X[best_idx].copy()
    best_fit = fit[best_idx].copy()

    stagnation_counter = 0

    # Main Loop Of The HHO
    for t in range(max_iter):
        # Escaping energy of the prey
        E = 2 * (1 - t / max_iter) * cp.random.uniform(-1, 1)
        noise_scale = 1.0 + (stagnation_counter / stagnation_limit)

        for i in range(N):
            if cp.abs(E) >= 1:  # Exploration phase
                q = cp.random.uniform()
                if q >= 0.5:
                    X[i] = cp.random.uniform(lb, ub, size=dim)
                else:
                    X[i] = best_pos - cp.random.uniform() * (ub - lb)
            else:  # Exploitation phase
                lf_step = levy_gpu(dim) * noise_scale
                X[i] += cp.random.uniform() * (best_pos - X[i]) + lf_step

            # Boundary clipping
            X[i] = cp.clip(X[i], lb, ub)

        # Fitness evaluation
        fit = cp.array([fitness_func(cp.asnumpy(X[i])) for i in range(N)])
        min_idx = int(cp.argmin(fit))

        if fit[min_idx] < best_fit:
            best_fit = fit[min_idx]
            best_pos = X[min_idx].copy()
            stagnation_counter = 0
        else:
            stagnation_counter += 1

        # Diversity restore
        if stagnation_counter >= stagnation_limit:
            reset_rate = min(max_reset_rate, 0.2 + stagnation_counter / (2 * stagnation_limit))
            reset_count = max(1, int(reset_rate * N))
            rand_idx = cp.random.choice(N, reset_count, replace=False)
            X[rand_idx] = cp.random.uniform(lb, ub, (reset_count, dim))
            stagnation_counter = 0

        # Logging
        print(f"Iter {t+1}/{max_iter} | Best Fitness: {float(best_fit):.6e}")

    return cp.asnumpy(best_pos), float(best_fit)

best_pos, best_fit = compressed_hho_adaptive(
    sphere_function, lb=-10, ub=10, dim=30,
    N=30, max_iter=500
)
print("\nBest Position:", best_pos)
print("Best Fitness:", best_fit)


# LSTM

## Preprocessing

- normalization
- min max scaler

---

## Training

- train dataset
- validation dataset
- test dataset

---

## Confusion matrix

- plot the matrix

---

## Saving the model

- save with .h5


# Compressed Version of HHO

In the original HHO, exploitation involves precise moves based on distance to the prey, escape factors (J), and Levy flights to escape local optima. It uses two Rapid Dive steps with separate fitness tests, requiring temporary storage of multiple positions (X1, X2, X_new…) and comparisons.  

In the compressed version, the goal is to remove extra storage and comparisons. Instead of fully executing the Rapid Dive and Soft Besiege formulas, it applies a single combined step: move toward `best_pos` plus Gaussian noise. This noise (`cp.random.normal`) acts like Levy flights or random diversity, but runs simpler and faster on GPU, reducing both GPU memory usage and fitness function calls.

# Fitness function and required functions


In [ ]:
def create_lstm_model(lstm_units1, lstm_units2, dropout1, dropout2, lr)
  pass

def train_and_validate(model, batch_size):
  pass


In [ ]:
# Our new fitness functin

def fitness_function(params):
  # params: [lstm_units1, lstm_units2, dropout1, dropout2, learning_rate, batch_size]
  lstm_units1 = int(params[0])
  lstm_units2 = int(params[1])
  dropout1 = params[2]
  dropout2 = params[3]
  learning_rate = params[4]
  batch_size = int(params[5])


  model = create_lstm_model(lstm_units1, lstm_units2, dropout1, dropout2, learning_rate)

  # Train the model
  acc = train_and_validate(model, batch_size)

  return -acc


# Implement HHO in LSTM